In [9]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()

X , y = data.data, data.target

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X_train.shape

torch.Size([16512, 8])

In [15]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [20]:
model = nn.Sequential(
    nn.Linear(8,24),
    nn.ReLU(),
    nn.Linear(24,12),
    nn.ReLU(),
    nn.Linear(12,6),
    nn.ReLU(),
    nn.Linear(6,1)
)

In [21]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [23]:
!pip install torchsummary

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

from torchsummary import summary

summary(model, (8,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 24]             216
              ReLU-2                   [-1, 24]               0
            Linear-3                   [-1, 12]             300
              ReLU-4                   [-1, 12]               0
            Linear-5                    [-1, 6]              78
              ReLU-6                    [-1, 6]               0
            Linear-7                    [-1, 1]               7
Total params: 601
Trainable params: 601
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [28]:
epochs = 500
losses = []

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss {loss.item():.4f}')

Epoch [10/500], Loss 1.3409
Epoch [20/500], Loss 1.3405
Epoch [30/500], Loss 1.3402
Epoch [40/500], Loss 1.3400
Epoch [50/500], Loss 1.3398
Epoch [60/500], Loss 1.3396
Epoch [70/500], Loss 1.3394
Epoch [80/500], Loss 1.3393
Epoch [90/500], Loss 1.3392
Epoch [100/500], Loss 1.3391
Epoch [110/500], Loss 1.3390
Epoch [120/500], Loss 1.3389
Epoch [130/500], Loss 1.3389
Epoch [140/500], Loss 1.3388
Epoch [150/500], Loss 1.3388
Epoch [160/500], Loss 1.3387
Epoch [170/500], Loss 1.3387
Epoch [180/500], Loss 1.3387
Epoch [190/500], Loss 1.3387
Epoch [200/500], Loss 1.3387
Epoch [210/500], Loss 1.3387
Epoch [220/500], Loss 1.3386
Epoch [230/500], Loss 1.3386
Epoch [240/500], Loss 1.3386
Epoch [250/500], Loss 1.3386
Epoch [260/500], Loss 1.3386
Epoch [270/500], Loss 1.3386
Epoch [280/500], Loss 1.3386
Epoch [290/500], Loss 1.3386
Epoch [300/500], Loss 1.3386
Epoch [310/500], Loss 1.3385
Epoch [320/500], Loss 1.3385
Epoch [330/500], Loss 1.3385
Epoch [340/500], Loss 1.3385
Epoch [350/500], Loss 1

In [34]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    # y_test_tensor = torch.tensor(y_test.values,dtype=torch.float32).unsqueeze(1)
    mse = F.mse_loss(y_pred, y_test)
    print(f'Mean Squared Error: {mse.item()}')

Mean Squared Error: 1.3121870756149292


C:\Users\DHAIRYA\AppData\Local\Temp\ipykernel_24368\151406636.py:5: UserWarning: Using a target size (torch.Size([4128])) that is different to the input size (torch.Size([4128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse = F.mse_loss(y_pred, y_test)


In [42]:
torch.save(model.state_dict(), 'model.pth')

In [40]:
import pickle

In [ ]:
# torch.save(model, 'model.h5')

In [46]:
load_model = nn.Sequential(
	nn.Linear(8,24),
	nn.ReLU(),
	nn.Linear(24,12),
	nn.ReLU(),
	nn.Linear(12,6),
	nn.ReLU(),
	nn.Linear(6,1)
)

load_model.load_state_dict(torch.load('model.pth'))
load_model.eval()

C:\Users\DHAIRYA\AppData\Local\Temp\ipykernel_24368\263177812.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model.load_state_dict(torch.load('model.pth'))


Sequential(
  (0): Linear(in_features=8, out_features=24, bias=True)
  (1): ReLU()
  (2): Linear(in_features=24, out_features=12, bias=True)
  (3): ReLU()
  (4): Linear(in_features=12, out_features=6, bias=True)
  (5): ReLU()
  (6): Linear(in_features=6, out_features=1, bias=True)
)

In [56]:
sample = X_test[1].unsqueeze(0)

load_model.to(device)
sample = sample.to(device)

with torch.no_grad():
    y_pred = load_model(sample)
    print(f'Predicted Price: {y_pred.item()}')

Predicted Price: 2.0670742988586426
